# Day 7

There are lot of rules about what types of bags must be contained by which bags.  

### Part 1
Determine how many *different colors of bags* could contain a ***shiny gold*** bag.

### Part 2
Determine how many *bags* must be contained by a ***shiny gold*** bag.

#### Read and Parse Input

In [1]:
f = open('day7.txt')
bags = f.readlines()
bags = [bag.strip() for bag in bags]
f.close()

In [2]:
bags[:5]

['posh brown bags contain 5 dim coral bags, 1 plaid blue bag, 2 faded bronze bags, 2 light black bags.',
 'vibrant lime bags contain 3 dull lavender bags, 3 dim crimson bags, 3 mirrored lavender bags, 2 muted cyan bags.',
 'clear olive bags contain 1 wavy gold bag, 4 dim lime bags, 3 dull tomato bags, 5 dark turquoise bags.',
 'dark purple bags contain 5 striped tan bags, 5 bright cyan bags, 3 dark indigo bags.',
 'posh maroon bags contain 3 bright salmon bags.']

**Sample 1**  
Uncomment this block to use this sample instead of the actual data.  

This sample should show an answer of "4 colors of bags" for Part 1 and "shiny gold contains 32 bags" for Part 2.

In [3]:
# sample = ['light red bags contain 1 bright white bag, 2 muted yellow bags.',
# 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.',
# 'bright white bags contain 1 shiny gold bag.',
# 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.',
# 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.',
# 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.',
# 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.',
# 'faded blue bags contain no other bags.',
# 'dotted black bags contain no other bags.']

# bags = sample

In [4]:
def parseBag(s):
    bag, contains = s.split('contain ')
    
    bag = bag.split(' ')
    bag = ' '.join(bag[:-2])
    
    contains = contains.split(', ')
    inside = []
    for thing in contains:
        sp_thing = thing.split(' ')
        if sp_thing[0] == 'no':
            continue
        count = int(sp_thing[0])
        name = sp_thing[1] + ' ' + sp_thing[2]
        inside.append([count, name])
    
    return (bag, inside)

In [5]:
for bag in bags[:3]:
    print(parseBag(bag))

('posh brown', [[5, 'dim coral'], [1, 'plaid blue'], [2, 'faded bronze'], [2, 'light black']])
('vibrant lime', [[3, 'dull lavender'], [3, 'dim crimson'], [3, 'mirrored lavender'], [2, 'muted cyan']])
('clear olive', [[1, 'wavy gold'], [4, 'dim lime'], [3, 'dull tomato'], [5, 'dark turquoise']])


### Build dicts to hold three relationships:
1. containments in goodBagDict
2. parent to child is in parentDict
3. child to parent is in childDict

I don't think I actually use parentDict...but I think I "sort of" needed it to make childDict.

In [6]:
goodBags = [parseBag(bag) for bag in bags]
goodBagDict = {k:v for k, v in goodBags}

In [7]:
# Get list of parents and dict where parents are keys
parents = [gb[0] for gb in goodBags]
parentDict= {p:[] for p in parents}
for gb in goodBags:
    for c in gb[1]:
        parentDict[gb[0]].append(c[1])

In [8]:
# Get list of children and dict where children are keys
children = []
for p in parentDict:
    for c in parentDict[p]:
        children.append(c)
children = list(set(children))
childDict = {c:[] for c in children}
for gb in goodBags:
    for c in gb[1]:
        childDict[c[1]].append(gb[0])

In [9]:
allBags = list(set(children + parents))

## Part 1
How many different colored bags could hold a ***shiny gold***?

In [10]:
toCheck = ['shiny gold']
holdsGold = [0 for i in range(len(allBags))]

while toCheck != []:
    newBags = [] # will hold all of the next layer up
    for bag in toCheck:
        holdsGold[allBags.index(bag)] = 1 # we've seen it now
        if bag in childDict:  # if it has any parents
            pars = childDict[bag]
            for par in pars:
                if holdsGold[allBags.index(par)] == 0: # and if those parents haven't been seen
                    newBags.append(par) # put the parent into the next round
    toCheck += newBags # add in the new bags
    for bag in toCheck: # remove anything we've already seen
        if holdsGold[allBags.index(bag)] == 1:
            toCheck.remove(bag)

In [11]:
# holdsGold will have a "1" for shiny gold, but it doesn't contain itself
sum(holdsGold) - 1

238

## Part 2
How many bags must be contained within a single shiny gold bag?  

This is sort of backwards of part 1 - part 1 was "how many colors of bag could *contain* a shiny gold", now its "how many bags are *contained by* a shiny gold".

**Sample 2**  
This cell below will overwrite all the necessary variables to be associated with this sample.  
Correct answer to Part 2 is "162 bags contained by one shiny gold".

In [12]:
# sample2 = ['shiny gold bags contain 2 dark red bags.',
# 'dark red bags contain 2 dark orange bags.',
# 'dark orange bags contain 2 dark yellow bags.',
# 'dark yellow bags contain 2 dark green bags.',
# 'dark green bags contain 2 dark blue bags.',
# 'dark blue bags contain 2 dark violet bags.',
# 'dark violet bags contain no other bags.']

# goodBags = [parseBag(bag) for bag in sample2]
# parents = [gb[0] for gb in goodBags]
# parentDict= {p:[] for p in parents}
# for gb in goodBags:
#     for c in gb[1]:
#         parentDict[gb[0]].append(c[1])
# children = []
# for p in parentDict:
#     for c in parentDict[p]:
#         children.append(c)
# children = list(set(children))
# childDict = {c:[] for c in children}
# for gb in goodBags:
#     for c in gb[1]:
#         childDict[c[1]].append(gb[0])
# allBags = list(set(children + parents))
# goodBagDict = {k:v for k, v in goodBags}

##### Yay recursion

In [13]:
def getContains(bag):
    kids = goodBagDict[bag]
    total = 0
    if kids == []:
        return 0
    for kid in kids:
        total += kid[0]*(getContains(kid[1]) + 1)
    return total

In [14]:
getContains('shiny gold')

82930